In [2]:
import hnswlib
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [3]:
name = 'glove-100-angular'
nameFull = name +'-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (1183514, 100)
testDataset :  (10000, 100)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create sclearn ann***

In [4]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] 
    num_elements = datasetImages.shape[0] 
    time_start = perf_counter()
    index = indexMethod(space = 'l2', dim = d)
    index.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
    ids = np.arange(num_elements)
    index.add_items(datasetImages, ids)
    index.set_ef(120)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(hnswlib.Index, datasetTrainImages)

Took 234.237 seconds


In [5]:
# (min, max) = createIndexNumerous(createIndex, AnnoyIndex, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [6]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        index, distance = indexedStruct.knn_query(xq, k = 100)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(np.sqrt(distance[0]))
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 0.517 seconds


In [7]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n','max : ', max,)

In [8]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [9]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [10]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [11]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

660281 || 660281
3.8792 || 3.8792
1133489 || 93438
3.9021 || 3.8838
97478 || 1133489
3.9037 || 3.9021
995275 || 97478
3.9294 || 3.9037
207076 || 995275
3.9378 || 3.9294
989330 || 207076
3.9523 || 3.9378
846101 || 989330
3.955 || 3.9523
242797 || 846101
3.9568 || 3.955
402216 || 242797
3.9697 || 3.9568
926741 || 402216
3.978 || 3.9697


In [12]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.1705
0.6593
0.9983


In [13]:
print(indexes[0]) # will find the 100 nearest neighbors

[ 660281 1133489   97478  995275  207076  989330  846101  242797  402216
  926741  256447  914908  229951  956338  859959   56568  647843  987056
  900915  963392  100544  861914 1025986   80231  134899  294110  716135
  452141  264602  499677  528570  392021  482291  660541  583340 1014707
  939324  147983  530481  430077  645035  924263  689945  292551  876802
 1034304   94664  639351  870693  904716  455463  209031  402905 1134185
  896732  135950   82685  867359  783290   21236  273527 1169856  988080
  444778  491578  699013  380601  703678  290452 1059144  636153 1182920
  327720 1025415 1071246  557971  999311  281134  138387  427585  615918
  797936  645387  130366  567120  871872    7642  906022  471520  547334
  640074  529306  493889  801019  702274  462795  332959  276292  723274
  423010]


In [14]:
# draw_mnist(indexes, distances, datasetTrainImages)